## Homework 1

#### 1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)

1. Для онлайн-кинотеатров (Кинопоиск, Okko и т.д.): рекомендованный фильм не только купили/посмотрели, но и поставили высокую оценку (выше среднего рейтинга). Альтернативно - поставили плохую оценку (с рекомендательной системой что-то не так).
2. Аналог для музыки (VK, Яндекс.Музыка и т.д.): рекомендованная музыка не только прослушана, но и добавлена в свой плейлист / ей поделились и т.д.

Т.е. по первым двум реализовывается некая метрика "удовлетворенности" с дополнительной оценкой от юзера.

3. Модификация любых метрик для оценки покупки (Яндекс.Маркет, Ozon и т.д.): рекомендованный товар не куплен, но куплен товар-аналог (той же категории) после перехода по рекомендации, в т.ч. после вторичной рекомендации - из карточки первого рекомендованного товара. Первая рекомендация тогда может быть "засчитана" с более низким весом.
4. Модификация любых метрик для оценки покупки: оценивать не только сам факт покупки, но и возможный возврат рекомендованного товара после покупки (в минус рекомендательной системе).

5. Ранжирование: не рекомендовать более 1-2 товаров из одной категории (снижается эффективность общей подборки).

#### 2) Доделать все функции, где стоит комментарий "сделать дома"

1. hit rate at k
2. money_precision_at_k
3. recall_at_k
4. money_recall_at_k
5. map_k
6. *NDCG@k
7. reciprocal_rank

#### Подготовка

In [1]:
import pandas as pd
import numpy as np

In [2]:
recommended_list = np.array([143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]) # id товаров
bought_list = np.array([521, 32, 143, 991])

prices_recommended = np.array([400, 60, 40, 40, 90, 100, 120, 10, 300, 50])
prices_bought = np.array([120, 60, 400, 40])

# список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                            [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                            [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]
                      ] # юзер 3

**hit rate at k**

In [3]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [4]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)

In [5]:
hit_rate_at_k(recommended_list, bought_list)

1

**money_precision_at_k**

In [6]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [7]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [8]:
def money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [9]:
money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5)

0.6984126984126984

**recall_at_k**

In [10]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(bought_list)

In [11]:
def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)

In [12]:
recall_at_k(recommended_list, bought_list, k=8)

0.5

**money_recall_at_k**

In [13]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_bought = np.array(prices_bought)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    money_recall = np.dot(flags, prices_recommended).sum() / prices_bought.sum()
    
    return money_recall

In [14]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=8)

0.7096774193548387

**map_k**

In [15]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant + 1) for index_relevant in relevant_indexes])
    
    return sum_ / amount_relevant

In [16]:
ap_k(recommended_list, bought_list, k=5)

0.75

In [17]:
def map_k(recommended_lists, bought_lists, k=5):
    
    sum_ = sum([ap_k(recommended_list, bought_list, k=k) for recommended_list, bought_list in zip(recommended_lists, bought_lists)])
    
    result = sum_ / len(recommended_lists)
    
    return result

In [18]:
map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

***NDCG@k**

In [19]:
def dcg_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0

    sum_ = 0
    
    for i in relevant_indexes:
        
        discount = i + 1 if i <= 1 else np.log2(i + 1)
        sum_ += 1 / discount
    
    return sum_ / k

In [20]:
dcg_k(recommended_list, bought_list, k=5)

0.3

In [21]:
def ndcg_k(recommended_list, bought_list, k=5):
    
    ideal_dcg_k = dcg_k(bought_list, bought_list, k=k)
    result = dcg_k(recommended_list, bought_list, k=k) / ideal_dcg_k
    
    return result

In [22]:
ndcg_k(recommended_list, bought_list, k=4)

0.5701406500739014

**reciprocal_rank**

In [23]:
def reciprocal_rank(recommended_list, bought_list):
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    return 1 / (relevant_indexes[0] + 1)

In [24]:
reciprocal_rank(recommended_list, bought_list)

1.0

In [25]:
def mrr(recommended_lists, bought_lists):
    
    sum_ = sum([reciprocal_rank(recommended_list, bought_list) for recommended_list, bought_list in zip(recommended_lists, bought_lists)])
    
    result = sum_ / len(recommended_lists)
    
    return result

In [26]:
mrr(recommended_list_3_users, bought_list_3_users)

0.3333333333333333